In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [10]:
# 데이터 불러오기

# 1. 레시피, 식재료 딕셔너리 읽어오기 (100차원)
with open(r'data\recipe\recipe.pickle', 'rb') as file:
    recipe_dict = pickle.load(file)

with open(r'data\recipe\ingre.pickle', 'rb') as file:
    ingre_dict = pickle.load(file)

with open(r'data\recipe\category.pickle', 'rb') as file:
    category_dict = pickle.load(file)

rec_title = list(recipe_dict.keys()) # 키 리스트
rec_vec = [recipe_dict[rec_title[i]] for i in range(len(recipe_dict))]

# 2. 상호작용 딕셔너리 불러오기 
hos = pd.read_csv(r'data\med\disease_interaction.csv', encoding = 'cp949')
dis_dict= {hos['질병명']: {'권장영양소' : hos['권장영양소'], '주의영양소' : hos['주의영양소'], '권장식재료' : hos['권장식재료'], '주의식재료' : hos['주의식재료']} for _, hos in hos.iterrows()}

hos = pd.read_csv(r'data\med\med_interaction.csv', encoding = 'cp949')
med_dict= {hos['dname']: {'권장영양소' : hos['권장영양소'], '주의영양소' : hos['주의영양소'], '권장식재료' : hos['권장식재료'], '주의식재료' : hos['주의식재료']} for _, hos in hos.iterrows()}

In [14]:
# 상호작용 함수 : 약, 질병 -> 권장/주의 식재료/영양소
def my_interaction(med_list, dis_list):
    interaction = {'권장식재료' : [], '주의식재료' : [], '권장영양소' : [], '주의영양소' : []}
    # 약물 정보 반영
    for med in med_list :
        for inter in interaction.keys():
            if not pd.isna(med_dict[med][inter]):
                interaction[inter].extend([value.strip() for value in med_dict[med][inter].split(',')])
    # 질병 정보 반영
    for dis in dis_list :
        for inter in interaction.keys() :
            if not pd.isna(dis_dict[dis][inter]):
                interaction[inter].extend([value.strip() for value in dis_dict[dis][inter].split(',')])
    return interaction

# my_interaction(['가나플럭스정20/1100mg', '본비바정150mg' ], ['2형 당뇨병', '뇌졸중', '구내염'])
# => 권장식재료, 주의식재료, 권장영양소, 주의영양소 리스트가 나옴

In [26]:
def recommend(my_med_list, my_dis_list, my_food):
    my_inter_dict = my_interaction(my_med_list, my_dis_list) # 입력한 질병, 약물을 상호작용 식재료, 영양소 불러오기
    my_food_vec = category_dict[my_food] # 내가 선택한 카테고리의 벡터
    my_food_vec_recommend = my_food_vec

    # 권장 : + 0.01 , 주의 -0.01 배로 해줌
    for food in my_inter_dict['권장식재료']:
        if food in ingre_dict :
            my_food_vec_recommend += 0.01* ingre_dict[food]
    for food in my_inter_dict['주의식재료']:
        if food in ingre_dict :
            my_food_vec_recommend -=  0.01 * ingre_dict[food]
    sim = cosine_similarity([my_food_vec_recommend], rec_vec)[0]
    recommend_idx = np.argsort(sim)[::-1][:100]

    return [rec_title[i] for i in recommend_idx]

In [27]:
recommend(['가나플럭스정20/1100mg', '본비바정150mg' ], ['2형 당뇨병', '뇌졸중', '구내염'], '찌개')

['해물만두칼국수 만들기 -온라인수업 아이 간단 점심',
 '간식과 술안주에도 좋은 감자 해쉬브라운 만드는 법',
 '감자 스테이크',
 '감자잡채호떡',
 '[GS25 야참히어로] 참 쉬운 떡&감자 그라탕',
 '생감자 샐러드',
 '아이반찬으로 좋은 감자채전',
 '포실포실한 전자렌지 감자구이',
 '에어프라이어 요리 웨지감자 만드는법 만들기',
 '알감자 조림',
 '삶은 감자버터구이 간단한 간식',
 '고구마 감자전(투포테이토전 )&소세지~',
 '감자고구마옥수수 샐러드 만들기',
 '감자브로콜리조림',
 '통감자 버터구이 만드는법',
 '맛있는 별식 미역감자죽',
 '알감자 버터구이 휴게소 버터감자 만들기 감자 요리 간식',
 '햇감자로 만든~더블치즈포테이토피자',
 '다이어트 요리, 밀가루 없는 바삭한 감자채전',
 '늙은호박 한통이면 [호박전, 호감전, 호구전] 다 만들 수 있어요~',
 '생감자튀김 만드는 법 : 비오는 날에는 따끈따끈 감튀에 시원한 맥주가 짱이쥐 ~',
 '홍감자구이/햇감자구이:추억의 통감자구이',
 '아삭이고추 감자 튀김',
 '바삭바삭 감자칩',
 '고구마칩&버터감자칩',
 '감자전 만들기 바삭 쫀득 감자요리',
 '아코디언감자 에어프라이어에 굽기',
 '훈제오리 볶음밥',
 '간식, 전자레인지로 만든 아코디언 감자구이',
 '감자튀김 에어프라이어로 만드는법',
 '명란감자볶음!!!',
 '감자탕',
 '감자 소세지 조림 만들기',
 '감자칩 만드는법',
 '사과 샐러드 ( apple salad )',
 '치즈감자 만드는법 ♪ 간단간식',
 '에어프라이어 홈메이드 간식 회오리감자 쉽게 만들기!',
 '웨지 포테이토',
 '집에서 즐기는 웨지감자와 휴게소 통감자 구이',
 '감자전 만들기',
 '감자버터구이...',
 '휴게소에서 파는 감자버터구이 만들기',
 '집에서 간단하게 홈메이드 감자칩 만들기~',
 '[맥주안주] 오지치즈후라이만들기',
 '♥[신혼밥상] 버섯부추 베이컨말이',
 '바삭바삭 고소한 감자칩!',
 '알감자

In [28]:
recommend(['가나플럭스정20/1100mg',], [ '뇌졸중', '구내염'], '돈까스')

['감자전 비 오는날 더 맛있는 감자 스팸 전',
 '야식으로 고소한~ 감자채전',
 '[감자고로케] 초간단! 감자만 있으면 쉽게 완성하는 감자고로케!! 감자케~ 한입 크기 쏙쏙!! -양싸',
 '바삭하고 고소한 감자치즈볼',
 '감자채전 만들기. 초보자도 만들수 있는 바삭한 베이컨감자전.',
 '메추리알 핫도그',
 '감자요리 / 감자 스크램블 에그',
 '방학간식 - 식빵감자미니핫도그',
 '집밥TV) 감자채전 만들기 | 버터 넣을까 말까',
 '쫄깃한 이탈리안 수제비 -뇨끼',
 '치즈 감자전',
 '다이어트 간식, 밀가루 없이 만든 감자채전',
 '아이들 간식으로 술안주로 딱인 "뢰스티" 레시피',
 '간단)또르띠야 데 빠따따(스페인 오믈렛)',
 '초간편 통새우고로케',
 '여름에 먹는 햇감자요리 감자토스트 레시피',
 '뇨끼 만들기 이탈리아 감자요리 뇨끼 만드는 방법',
 '감자채전 백종원 새둥지 감자채전 감자전 내맘대로 레시피',
 '단호박프리타타 감자프리타타 전자레인지로 손쉽게~ (단호박오믈렛 /감자오믈렛)',
 '메인요리인 감자와 계란과 피자치즈를 활용해서 만든 [퓨전감자전] 입니다.',
 '새우볶음밥 소시지 얹어 맛있게~',
 '집에서 만드는 길거리 토스트',
 '내멋대로 감자프리타타♡냉파요리♡초간단이예요~',
 '감자채전 이맛에 반했네',
 '질리지 않는 담백한맛/ 감자 베이컨 팬케이크',
 '여름제철 감자요리/간단 브런치로 좋은 계란 감자채전',
 '독특한 감자전 만들기 감자채전 만드는법',
 '구황작물 감자오믈렛, 스페인 tortilla de patatas',
 '감자전 바삭하면서 맛있는 감자전',
 '감자채전 아침식사로 든든하게',
 '감자채전 만들기 / 새둥지전',
 '간단하지만 바삭 고소하고 맛있는 치즈 계란 감자채전/맥주안주/간식',
 '감자넣어 구수한 북어국 끓이기',
 '감자 북어국 참 간단하다',
 '[간단 자취요리] 콘치즈가 고로케 속에! 콘치즈 고로케 만들기',
 "스위스식 감자전 '뢰스티' 아이들 간식, 브런치 메뉴 강